In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
from torchvision.utils import save_image
import torchvision.transforms as transforms
from torchvision import datasets, models, transforms
import torchvision

import os
import sys
sys.path.insert(0, "../utils")
import numpy as np
import matplotlib
from matplotlib import pyplot as plt
from tqdm import tqdm

import src.datasets.cityscapes_loader as cityscapes_loader
import utils.train_eval as train_eval
import importlib
import visualizations as vis

%load_ext autoreload
%autoreload 2



The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
importlib.reload(cityscapes_loader)

is_sequence = True

dataset_root_dir = "/home/nfs/inf6/data/datasets/cityscapes/"

train_ds = cityscapes_loader.cityscapesLoader(root=dataset_root_dir, split='train', img_size=(512, 1024), is_transform=True, is_sequence=is_sequence)
val_ds = cityscapes_loader.cityscapesLoader(root=dataset_root_dir, split='val', img_size=(512, 1024), is_transform=True, is_sequence=is_sequence)

train_loader = torch.utils.data.DataLoader(train_ds, batch_size=2, shuffle=True, drop_last=True)
valid_loader = torch.utils.data.DataLoader(val_ds, batch_size=1, shuffle=False, drop_last=True)

Found 2975 train images
Found 500 val images


In [6]:
from src.architectures.architecture_configs import *
import src.architectures.Temporal_UNET_Template as Temporal_UNET_Template
import utils.utils

encoder_blocks = SmallDeep_NetworkSize.encoder_blocks
decoder_blocks = SmallDeep_NetworkSize.decoder_blocks

config = Temporal_ResUNetConfig(
    encoder_blocks=encoder_blocks,
    decoder_blocks=decoder_blocks,
    temporal_cell= Conv2dGRUCell
    )

temp_unet = Temporal_UNET_Template.Temporal_UNet(config)

temp_unet_optim = torch.optim.Adam(temp_unet.parameters(), lr=3e-4)
criterion = nn.CrossEntropyLoss()

epochs=100
temp_unet_trainer = utils.train_eval.Trainer(
            temp_unet, temp_unet_optim, criterion,
            train_loader, valid_loader, "cityscapes", epochs,
            sequence=True, all_labels=20, start_epoch=17)

load_model = True
if load_model:
    temp_unet_trainer.load_model("cityscapes")

/home/user/sheludzk/CudaLab_Project/tboard_logs/Temporal_ResUNetConfig_Conv2dGRUCell/Layers4_InitDim16


In [7]:
import shutil
import os

@torch.no_grad()
def vis_seq(model, loader):
    visualizer = vis.CityscapesVisualizer()
    model.eval()
    for i, (imgs, targets) in enumerate(loader):
        imgs = imgs.cuda()
        preds = model(imgs)
        print(preds.shape)
        for i in range(preds.shape[0]):
            decoded_seq = get_decoded_img_seq(preds[i])

            for j in range(len(decoded_seq)):
                plt.imshow(decoded_seq[j])
                plt.show()

            break
        break
    return 

@torch.no_grad()
def save_vis_seq(model, loader, model_name="default"):
    if not os.path.exists("imgs"):
        os.makedirs("imgs")
    if not os.path.exists(f"imgs/{model_name}"):
        os.makedirs(f"imgs/{model_name}")

    visualizer = vis.CityscapesVisualizer()
    model.eval()
    for k, (imgs, targets) in enumerate(loader):
        if not os.path.exists(f"imgs/{model_name}/{k}"):
            os.makedirs(f"imgs/{model_name}/{k}")
        if not os.path.exists(f"imgs/{model_name}/{k}/original"):
            os.makedirs(f"imgs/{model_name}/{k}/original")
        if not os.path.exists(f"imgs/{model_name}/{k}/predicted"):
            os.makedirs(f"imgs/{model_name}/{k}/predicted")
        imgs = imgs.cuda()
        preds = model(imgs)
        print(preds.shape)
        for i in range(preds.shape[0]):

            decoded_seq = get_decoded_img_seq(preds[i])

            for j in range(len(decoded_seq)):
                torchvision.utils.save_image(torch.from_numpy(decoded_seq[j].transpose(2,0,1)), os.path.join(os.getcwd(), "imgs", f"{model_name}", f"{k}", "predicted", f"imgs_{j}.png"))

                mean = torch.tensor([0.485, 0.456, 0.406]).view(1, 3, 1, 1)
                std = torch.tensor([0.229, 0.224, 0.225]).view(1, 3, 1, 1)
                unnorm_imgs = imgs.cpu() * std + mean

                torchvision.utils.save_image(unnorm_imgs[i, j], os.path.join(os.getcwd(), "imgs", f"{model_name}", f"{k}", "original", f"imgs_{j}.png"))

        if k > 5:
            break

    return 

def get_decoded_img_seq(preds):
    result = []
    visualizer = vis.CityscapesVisualizer()
    predicted_class = torch.argmax(preds, dim=1)
    for j in range(preds.shape[0]):
        decoded_pred = visualizer.decode_segmap(predicted_class[j].cpu().numpy())
        result.append(decoded_pred)
        #torchvision.utils.save_image(torch.from_numpy(decoded_pred.transpose(2,0,1)), os.path.join(os.getcwd(), "imgs", "training", f"imgs_{j}.png"))
    return result

save_vis_seq(temp_unet, valid_loader)

torch.Size([1, 12, 20, 512, 1024])
torch.Size([1, 12, 20, 512, 1024])
torch.Size([1, 12, 20, 512, 1024])
torch.Size([1, 12, 20, 512, 1024])
torch.Size([1, 12, 20, 512, 1024])
torch.Size([1, 12, 20, 512, 1024])
torch.Size([1, 12, 20, 512, 1024])


In [2]:
import imageio
import PIL
import os

def create_gifs(model_name="default", mode="side-by-side", transparency=0.5, fps=8):

    allowed_modes = ["side-by-side", "overlay"]
    if mode not in allowed_modes:
        raise ValueError(f"mode must be one of {alloud_modes}")
    imgs_root=f"imgs/{model_name}"
    dirlist = [ item for item in os.listdir(imgs_root) if os.path.isdir(os.path.join(imgs_root, item)) ]
    # remove "gifs"-folder from dirlist
    dirlist = [item for item in dirlist if item != "gifs"]

    if not os.path.exists(f"imgs/{model_name}/gifs"):
        os.makedirs(f"imgs/{model_name}/gifs")



    if mode == "side-by-side":
        for i in range(len(dirlist)):
            images = []
            for j in range(12):
                original = PIL.Image.open(f"imgs/{model_name}/{dirlist[i]}/original/imgs_{j}.png")
                prediction = PIL.Image.open(f"imgs/{model_name}/{i}/predicted/imgs_{j}.png")

                (width1, height1) = original.size
                (width2, height2) = prediction.size

                result_width = width1 + width2
                result_height = max(height1, height2)

                result = PIL.Image.new('RGB', (result_width, result_height))
                result.paste(im=original, box=(0, 0))
                result.paste(im=prediction, box=(width1, 0))
                images.append(result)
            imageio.mimsave(f"imgs/{model_name}/gifs/{i}.gif", images, fps=fps)

    elif mode == "overlay":
        for i in dirlist:
            images = []
            for j in range(12):
                background = PIL.Image.open(f"imgs/{model_name}/{i}/original/imgs_{j}.png")
                foreground  = PIL.Image.open(f"imgs/{model_name}/{i}/predicted/imgs_{j}.png")
                foreground.putalpha(int(255*(1-transparency))) 
                background.paste(foreground, (0, 0), mask=foreground)
                images.append(background)
            imageio.mimsave(f"imgs/{model_name}/gifs/{i}.gif", images, fps=fps)

create_gifs("default", mode="overlay", transparency=0.45, fps=8)